In [1]:
import pandas as pd

In [2]:
sell = {1300.05: 100, 1300.04: 30, 1300.03: 20, 1300.02: 10, 1300.01: 5, 1300.0: 100}
buy = {1299.99: 2, 1299.98: 10, 1299.97: 50, 1299.96: 100, 1299.95: 2}

In [3]:
buy = pd.DataFrame.from_dict(buy,orient='index',columns=['lots']).reset_index().rename(columns={'index':'price'})
sell = pd.DataFrame.from_dict(sell,orient='index',columns=['lots']).reset_index().rename(columns={'index':'price'})
buy = buy.sort_values(by='price',ascending=False).reset_index(drop=True)
sell = sell.sort_values(by='price',ascending=True).reset_index(drop=True)

In [4]:
df = pd.read_csv('C:\\Users\\10114\\Desktop\\operate.csv')

In [5]:
df

,op,order_type,price,lots
0,buy,limit order,1299.98,10
1,sell,limit order,1300.03,10
2,buy,market order,NaN,5
3,sell,market order,NaN,10
4,buy,market order,NaN,2
5,buy,limit order,1299.99,2
6,buy,market order,NaN,100
7,sell,limit order,1300.00,10
8,sell,limit order,1300.01,30
9,sell,market order,NaN,30


In [6]:
buy

,price,lots
0,1299.99,2
1,1299.98,10
2,1299.97,50
3,1299.96,100
4,1299.95,2


In [7]:
sell

,price,lots
0,1300.00,100
1,1300.01,5
2,1300.02,10
3,1300.03,20
4,1300.04,30
5,1300.05,100


In [8]:
for i in df.itertuples():
    print(i)

Pandas(Index=0, op='buy', order_type='limit order', price=1299.98, lots=10)
Pandas(Index=1, op='sell', order_type='limit order', price=1300.03, lots=10)
Pandas(Index=2, op='buy', order_type='market order', price=nan, lots=5)
Pandas(Index=3, op='sell', order_type='market order', price=nan, lots=10)
Pandas(Index=4, op='buy', order_type='market order', price=nan, lots=2)
Pandas(Index=5, op='buy', order_type='limit order', price=1299.99, lots=2)
Pandas(Index=6, op='buy', order_type='market order', price=nan, lots=100)
Pandas(Index=7, op='sell', order_type='limit order', price=1300.0, lots=10)
Pandas(Index=8, op='sell', order_type='limit order', price=1300.01, lots=30)
Pandas(Index=9, op='sell', order_type='market order', price=nan, lots=30)
Pandas(Index=10, op='buy', order_type='market order', price=nan, lots=20)
Pandas(Index=11, op='buy', order_type='limit order', price=1299.98, lots=20)
Pandas(Index=12, op='buy', order_type='market order', price=nan, lots=50)
Pandas(Index=13, op='sell', 

In [9]:
opr = df.itertuples()

In [10]:
for one_step in opr:
    if one_step.op == 'buy':
        # 想买
        ask_price = one_step.price
        ask_lots = one_step.lots
        if one_step.order_type == 'limit order':
            # 想买，限价单
            if sell['price'][0] > ask_price:
                # 没有可以匹配的订单，加入buy表
                price_index = buy[buy['price']==ask_price].index.tolist()
                if len(price_index) == 0:
                    buy = buy.append({'price' : ask_price , 'lots' : ask_lots} , ignore_index=True)
                else:
                    buy.iloc[price_index[0], -1] += ask_lots

            else:
                # 有可以匹配的订单
                for i in list(sell.index):
                    if sell.iloc[i, 0] > ask_price or ask_lots <= 0:
                        break
                    if ask_lots >= sell.iloc[i, -1]:
                        ask_lots -= sell.iloc[i, -1]
                        sell.iloc[i, -1] = 0
                    else:
                        sell.iloc[i, -1] -= ask_lots
                        ask_lots = 0

                if ask_lots > 0:
                    print('市场上所有可买均买入')
                    # 因为是限价单，所以需把剩余加入buy表中
                    price_index = buy[buy['price']==ask_price].index.tolist()
                    if len(price_index) == 0:
                        buy = buy.append({'price' : ask_price , 'lots' : ask_lots} , ignore_index=True)
                    else:
                        buy.iloc[price_index[0], -1] += ask_lots

        elif one_step.order_type == 'market order':
            # 想买，市价单
            if sell['price'][0] > ask_price:
                # 没有可以匹配的订单，加入buy表
                price_index = buy[buy['price']==ask_price].index.tolist()
                if len(price_index) == 0:
                    # 列表为空
                    buy = buy.append({'price' : ask_price , 'lots' : ask_lots} , ignore_index=True)
                else:
                    buy.iloc[price_index[0], -1] += ask_lots

            else:
                # 可以匹配
                for i in list(sell.index):
                    if ask_lots <= 0:
                        break
                    if ask_lots >= sell.iloc[i, -1]:
                        ask_lots -= sell.iloc[i, -1]
                        sell.iloc[i, -1] = 0
                    else:
                        sell.iloc[i, -1] -= ask_lots
                        ask_lots = 0

                if ask_lots > 0:
                    print('市场上所有可买均买入')
                    # 市价单，理应raise error

        else:
            print('第{}行（个）操作类型有误'.format(one_step.Index+1))


    elif one_step.op == 'sell':
        # 想卖
        sell_price = one_step.price
        sell_lots = one_step.lots
        if one_step.order_type == 'limit order':
            # 想卖，限价单
            if buy['price'].iloc[0] < sell_price:
                # 没有可以匹配的订单，加入sell表
                price_index = sell[sell['price']==sell_price].index.tolist()
                if len(price_index) == 0:
                    print(1)
                    # 列表为空
                    print(type(sell_lots))
                    sell = sell.append([{'price' : sell_price , 'lots' : int(sell_lots)}] , ignore_index=True)
                else:
                    #print(price_index)
                    sell.iloc[price_index[0], -1] += sell_lots

            else:
                # 有可以匹配的订单
                for i in list(buy.index):
                    if buy['price'][i] < sell_price or sell_lots <= 0:
                        break
                    if sell_lots >= buy.iloc[i, -1]:
                        sell_lots -= buy.iloc[i, -1]
                        buy.iloc[i, -1] = 0
                    else:
                        buy.iloc[i, -1] -= sell_lots
                        sell_lots = 0

                if sell_lots > 0:
                    print('市场上所有可卖均卖出')
                    # 限价单 把没买加入sell表
                    price_index = sell[sell['price']==sell_price].index.tolist()
                    if len(price_index) == 0:
                        #print(sell_lots)
                        #sell_lots = sell_lots.astype(int)
                        sell = sell.append({'price' : sell_price , 'lots' : sell_lots} , ignore_index=True)
                    else:
                        sell.iloc[price_index[0], -1] += sell_lots

        elif one_step.order_type == 'market order':
            # 想卖，市价单
            if buy['price'].iloc[0] < sell_price:
                # 没有可以匹配的订单，加入sell表
                price_index = sell[sell['price']==sell_price].index.tolist()
                if len(price_index) == 0:
                    sell = sell.append({'price': sell_price, 'lots': sell_lots}, ignore_index=True)
                else:
                    sell.iloc[price_index[0], -1] += sell_lots

            else:
                # 可以匹配
                for i in list(buy.index):
                    if sell_lots <= 0:
                        break
                    if sell_lots >= buy.iloc[i, -1]:
                        sell_lots -= buy.iloc[i, -1]
                        buy.iloc[i, -1] = 0
                    else:
                        buy.iloc[i, -1] -= sell_lots
                        sell_lots = 0

                if sell_lots > 0:
                    print('市场上所有可卖均卖出')
                    # 市价单，理应raise error

        else:
            print('第{}行（个）操作类型有误'.format(one_step.Index+1))

    buy = buy[~buy['lots'].isin([0])]
    sell = sell[~sell['lots'].isin([0])]

    buy['lots'] = buy['lots'].astype(float)
    sell['lots'] = sell['lots'].astype(float)

    # 每次操作后重新排列两张表
    buy = buy.sort_values(by='price',ascending=False).reset_index(drop=True)
    sell = sell.sort_values(by='price',ascending=True).reset_index(drop=True)
    sell['price'].astype(int)

    print('第{}行（个）操作后'.format(one_step.Index+1))
    print('------------------------------')
    print('buy表：')
    print(buy)
    print('------------------------------')
    print('sell表：')
    print(sell)
    print('------------------------------')
    print('\n\n')

第1行（个）操作后
------------------------------
buy表：
     price   lots
0  1299.99    2.0
1  1299.98   20.0
2  1299.97   50.0
3  1299.96  100.0
4  1299.95    2.0
------------------------------
sell表：
     price   lots
0  1300.00  100.0
1  1300.01    5.0
2  1300.02   10.0
3  1300.03   20.0
4  1300.04   30.0
5  1300.05  100.0
------------------------------



第2行（个）操作后
------------------------------
buy表：
     price   lots
0  1299.99    2.0
1  1299.98   20.0
2  1299.97   50.0
3  1299.96  100.0
4  1299.95    2.0
------------------------------
sell表：
     price   lots
0  1300.00  100.0
1  1300.01    5.0
2  1300.02   10.0
3  1300.03   30.0
4  1300.04   30.0
5  1300.05  100.0
------------------------------



第3行（个）操作后
------------------------------
buy表：
     price   lots
0  1299.99    2.0
1  1299.98   20.0
2  1299.97   50.0
3  1299.96  100.0
4  1299.95    2.0
------------------------------
sell表：
     price   lots
0  1300.00   95.0
1  1300.01    5.0
2  1300.02   10.0
3  1300.03   30.0
4  1300.04 